In [1]:
import json
import pandas as pd

with open('/Users/adityachawra/Downloads/Startup/DataBase/Data/MF/investment_options-master/amfi_data_fetcher/nav_equity_quant_json.json') as file:
    data = json.load(file)
    

In [7]:
from langchain.schema import Document

documents = []

for fund in data:
    content = f"""
    scheme_code: {fund['scheme_code']}
    sharpe_ratio: {fund['sharpe_ratio']}
    sortino_ratio: {fund['sortino_ratio']}
    beta: {fund['beta']}
    alpha_annualized: {fund['alpha_annualized']}
    treynor_ratio: {fund['treynor_ratio']}
    """
    documents.append(Document(page_content=content))
        

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})  # top 3 relevant funds


/var/folders/j3/btykbjss2l15b55qw91h85hc0000gn/T/ipykernel_53509/1450670707.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/adityachawra/Downloads/Startup/AI/RAG/ragtest/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os

load_dotenv()  # This loads variables from .env into your environment

# Now OPENAI_API_KEY is available as an environment variable

from langchain_google_genai import ChatGoogleGenerativeAI

# Setup Gemini LLM


llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GEMINI_API_KEY"),
    temperature=0
)



rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)


In [31]:
user_query = input("Enter your query about mutual funds: ")


full_query = f"""
This is a financial advisory system that recommends mutual funds based on user queries.
Answer only according the to context the documents provided through rag_chain.

User Query: {user_query}
Explain why each recommended fund is a good fit, not in a technical way but how it suits their needs.
Provide a detailed explanation of the recommended funds, including their performance metrics and how they align with the user's query.
Use the following format for your response:
1. Fund Name: [Name of the fund]
   - Explanation: [Detailed explanation of why this fund is recommended]
2. Fund Name: [Name of the fund]
   - Explanation: [Detailed explanation of why this fund is recommended]
and so on for each recommended fund.
help them with their query and provide a comprehensive answer.
"""

result = rag_chain(full_query)

print(result['result'])

Based on your preference for higher risk bets and a decent return, I recommend the following mutual fund:

1.  **Fund Name: Scheme 144836**
    *   **Explanation:** This fund appears to be a strong candidate for your investment goals. You mentioned being comfortable with some higher risk, and this fund's performance metrics suggest it has historically delivered excellent returns for the level of risk it takes.
        *   **Why it suits your needs:**
            *   **Higher Risk Tolerance:** While its market sensitivity (`beta` of 0.81) is slightly higher than some other options, it indicates that it might experience slightly larger movements in line with the overall market. This aligns with your comfort level for "higher risk bets."
            *   **Decent Return Potential:** This fund truly shines in its ability to generate returns. Its `alpha_annualized` of 0.34 is exceptionally high, meaning it has consistently outperformed what would be expected given its risk level. In simpler 

{'query': '\nThis is a financial advisory system that recommends mutual funds based on user queries.\nAnswer only according the to context the documents provided through rag_chain.\n\nUser Query: Suggest scheme code form high sortino ratio and lower beta.\nExplain why each recommended fund is a good fit.\n', 'result': "Based on your criteria for a high Sortino ratio and lower beta, the following scheme codes are recommended:\n\n*   **Scheme Code: 144545**\n    *   **Why it's a good fit:** It has the highest Sortino ratio (7.269) among the provided options, indicating excellent risk-adjusted returns, particularly in managing downside risk. It also has the lowest beta (0.716), suggesting lower volatility compared to the overall market.\n*   **Scheme Code: 144544**\n    *   **Why it's a good fit:** Similar to 144545, it also has the highest Sortino ratio (7.269), signifying strong performance relative to downside risk. Its beta (0.716) is also the lowest, indicating less sensitivity to ma